In [1]:
%load_ext autoreload
%autoreload 2

from tricount import api

def get_data():
    data, dict = api.fetch_data()
    data = api.build_data(data)
    detail = api.split_data(data, dict)
    postes, result = api.concat_data(detail, dict)
    return detail, postes, result

detail, postes, result = get_data()


    # --------------------------------- #
    # Connection to Google Sheet        #
    # and extract Spreadsheets          #
    #  - exported tricount data         #
    #  - category dictonnary            #
    # --------------------------------- #
    
 -- data fetched
(1247, 14)
Index(['Titre', 'Montant', 'Devise', 'Taux de change',
       'Montant dans la devise du tricount (EUR)', 'Type de transaction',
       'Catégorie', 'Payé par', 'Payé par Lucie', 'Payé par Vincent',
       'Impacté à Lucie', 'Impacté à Vincent', 'Date & heure',
       'URL des images'],
      dtype='object')
 -- dict fetched 
{'Alimentation': 'Quotidien', 'Shopping': 'Achats', 'Logement': 'Achats', 'Loyer & charges': 'Quotidien', 'Divertissement': 'Extra', 'Sport': 'Loisir', 'Restaurant & bar': 'Extra', 'Investissement': 'Investissement', 'Transport': 'Quotidien', 'Week-end': 'Loisir', 'Soin de santé': 'Quotidien', 'Livres': 'Achats', 'Activités': 'Loisir', 'Boulangerie': 'Quotidien', 'Salaire': "Rentrée 

In [48]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

periods = ['2022 (mean)', '2023 January', '2023 February', '2023 March', '2023 April']
people = 'Vincent'

rows = {}
for period in periods:
    rows[period] = result[period][people]
df = pd.concat(rows, axis=1).transpose()


fig = go.Figure()
for metric in [m for m in df.columns if m in ['Revenus', 'Dépenses', 'Reste à vivre', 'Capital investi', 'Epargne']]:
    fig.add_trace(go.Scatter(x=df.index.tolist(), y=np.abs(df[metric].values.tolist()), name=metric))

fig.show()

In [52]:
df2 = df.transpose()

fig = go.Figure()
for metric in df2.columns:
    fig.add_trace(go.Bar(x=df2.index.tolist(), y=np.abs(df2[metric].values.tolist()), name=metric))

fig.show()